<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 3.6: Generators: Types
**Prev: [Object Oriented Programming](3.5_object_oriented_programming.ipynb)**<br>
**Next: [Introduction to FIRRTL](4.1_firrtl_ast.ipynb)**

## Motivation
스칼라는 강력한 형식의 프로그래밍 언어입니다. 이것은 양날의 검입니다. 한편으로 Python(동적 타입 언어)에서 컴파일 및 실행되는 많은 프로그램은 Scala에서 컴파일 시간에 실패합니다. 반면 스칼라에서 컴파일하는 프로그램은 유사한 Python 프로그램보다 런타임 오류가 훨씬 적습니다.

이 섹션에서 우리의 목표는 스칼라의 일급 시민으로서 타입에 익숙해지는 것입니다. 처음에는 생산성이 제한적이라고 느낄 수 있지만 곧 컴파일 시간 오류 메시지를 이해하는 방법과 더 많은 오류를 포착하기 위해 타입 시스템을 염두에 두고 프로그램을 설계하는 방법을 배우게 될 것입니다.


## Setup

In [1]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

Checking https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/chisel-iotesters_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/chisel-iotesters_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/chiseltest_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/chiseltest_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/dsptools_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/dsptools_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/firrtl-diagrammer_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/firrtl-diagrammer_2.12/maven-metadata.xml


path: String = "/home/parkdongho/dev/chisel-bootcamp-kr/source/load-ivy.sc"

In [2]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.tester.RawTester.test

---
# Static Types<a name="types-in-scala"></a>

## Types in Scala

Scala의 모든 객체에는 일반적으로 객체의 클래스인 타입이 있습니다.
몇 가지를 보겠습니다.

In [3]:
println(10.getClass)
println(10.0.getClass)
println("ten".getClass)

int
double
class java.lang.String


자신의 클래스를 선언하면 연결된 타입이 있습니다.

In [43]:
class MyClass {
    def myMethod = ???
}
println(new MyClass().getClass)

class ammonite.$sess.cmd42$Helper$MyClass


defined class MyClass

필수는 아니지만 **모든 함수 선언에 대한 입력 및 출력 타입을 정의**하는 것이 좋습니다. 이렇게 하면 스칼라 컴파일러가 함수의 부적절한 사용을 포착할 수 있습니다.

In [3]:
def double(s: String): String = s + s
// Uncomment the code below to test it
 double("hi")      // Proper use of double
 double(10)        // Bad input argument!
 double("hi") / 10 // Inproper use of double's output!

cmd3.sc:4: type mismatch;
 found   : Int(10)
 required: String
 val res3_2 = double(10)        // Bad input argument!
                     ^cmd3.sc:5: value / is not a member of String
 val res3_3 = double("hi") / 10 // Inproper use of double's output!
                           ^Compilation Failed

: 

아무 것도 반환하지 않는 함수는 `Unit` 타입을 반환합니다.

In [4]:
var counter = 0
def increment(): Unit = {
    counter += 1
}
increment()

counter: Int = 1
defined function increment

## Scala vs. Chisel Types<a name="scala-vs-chisel-types"></a>

요약: 모듈 2.2에서는 Chisel 타입과 Scala 타입의 차이점에 대해 논의했습니다. 예를 들어
```scala
val a = Wire(UInt(4.W))
a := 0.U
```
`0`U'는 'UInt' 타입(Chisel 타입)이기 때문에 합법적이지만,
```scala
val a = Wire(UInt(4.W))
a := 0
```
0은 'Int' 타입(Scala 타입)이기 때문에 불법입니다.

이것은 `Boolean`(Scala 타입)과 구별되는 Chisel 타입인 `Bool`에도 해당됩니다.
```scala
val bool = Wire(Bool())
val boolean: Boolean = false
// legal
when (bool) { ... }
if (boolean) { ... }
// illegal
if (bool) { ... }
when (boolean) { ... }
```

실수를 해서 `UInt`와 `Int` 또는 `Bool`과 `Boolean`을 혼동하면 일반적으로 Scala 컴파일러가 이를 잡아줍니다. 이것은 스칼라의 정적 타이핑 때문입니다. 컴파일 타임에 컴파일러는 Chisel과 Scala 타입을 구별할 수 있으며 `if()`가 `Boolean`을 예상하고 `when()`이 `Bool`을 예상한다는 것도 이해할 수 있습니다.


## Scala Type Coercion<a name="type-coercion"></a>

<!-- typeOf. Scala has a function called `typeOf[T]` which returns a type object for `T`. -->
<!-- This doesn't actually seem useful to Chisel users... -->

### asInstanceOf

`x.asInstanceOf[T]`는 객체 `x`를 `T` 타입으로 캐스트합니다. 주어진 객체를 `T` 타입으로 캐스팅할 수 없는 경우 예외가 발생합니다.

In [46]:
val x: UInt = 3.U
try {
  println(x.asInstanceOf[Int])
} catch {
  case e: java.lang.ClassCastException => println("As expected, we can't cast UInt to Int")
}

// But we can cast UInt to Data since UInt inherits from Data.
println(x.asInstanceOf[Data])

As expected, we can't cast UInt to Int
UInt<2>(3)


x: UInt = UInt<2>(3)


### Type Casting in Chisel

아래 코드는 주석을 제거하지 않고 실행하려고 하면 오류가 발생합니다. 뭐가 문제 야? 불법인 `SInt`에 `UInt`를 할당하려고 합니다.

Chisel에는 타입 캐스팅 기능 세트가 있습니다. 가장 일반적인 것은 'asTypeOf()'로 아래와 같습니다. 일부 Chisel 객체는 `asUInt()` 및 `asSInt()`뿐만 아니라 일부 다른 객체도 정의합니다.

아래 코드 블록에서 `//`를 제거하면 예제가 작동합니다.


In [6]:
class TypeConvertDemo extends Module {
    val io = IO(new Bundle {
        val in  = Input(UInt(4.W))
        val out = Output(SInt(4.W))
    })
    io.out := io.in.asTypeOf(io.out)
}

test(new TypeConvertDemo) { c =>
      c.io.in.poke(3.U)
      c.io.out.expect(3.S)
      c.io.in.poke(15.U)
      c.io.out.expect(-1.S)
}

Elaborating design...
Done elaborating.
test TypeConvertDemo Success: 0 tests passed in 2 cycles in 0.017976 seconds 111.26 Hz


defined class TypeConvertDemo

---
# Type Matching<a name="type-matching"></a>

## Match Operator
3.1에서 일치 연산자(===)가 도입되었음을 상기하십시오.
타입 일치는 타입 제네릭 generator를 작성하려고 할 때 특히 유용합니다. 다음 예는 `UInt` 또는 `SInt` 타입의 두 리터럴을 추가할 수 있는 "제너레이터"의 예를 보여줍니다. 이후 섹션에서는 타입 제네릭 generator 작성에 대해 자세히 설명합니다.

**참고: Scala에서 타입 제네릭 generator를 작성하는 훨씬 더 좋고 안전한 방법이 있습니다**.

In [7]:
class ConstantSum(in1: Data, in2: Data) extends Module {
    val io = IO(new Bundle {
        val out = Output(chiselTypeOf(in1)) // in case in1 is literal then just get its type
    })
    (in1, in2) match {
        case (x: UInt, y: UInt) => io.out := x + y
        case (x: SInt, y: SInt) => io.out := x + y
        case _ => throw new Exception("I give up!")
    }
}
println(getVerilog(dut = new ConstantSum(3.U, 4.U)))
println(getVerilog(dut = new ConstantSum(-3.S, 4.S)))
println(getVerilog(dut = new ConstantSum(3.U, 4.S)))

Elaborating design...
Done elaborating.
module ConstantSum(
  input        clock,
  input        reset,
  output [1:0] io_out
);
  assign io_out = 2'h3; // @[cmd6.sc 6:43]
endmodule

Elaborating design...
Done elaborating.
module ConstantSum(
  input        clock,
  input        reset,
  output [2:0] io_out
);
  assign io_out = 3'sh1; // @[cmd6.sc 7:43]
endmodule

Elaborating design...
[error] 	at ... ()
[error] 	at ammonite.$sess.cmd6$Helper$ConstantSum.<init>(cmd6.sc:8)
[error] 	at ammonite.$sess.cmd6$Helper.$anonfun$res6_3$1(cmd6.sc:13)
[error] 	at ... ()
[error] 	at ... (Stack trace trimmed to user code only. Rerun with --full-stacktrace to see the full stack trace)


: 

Chisel 타입은 일반적으로 값이 일치하지 않아야 함을 기억하는 것이 좋습니다.
Scala의 일치는 회로 정교화(elaboration) 중에 실행되지만 아마도 원하는 것은 사후 정교화 비교입니다.
다음은 구문 오류를 제공합니다.

In [48]:
class InputIsZero extends Module {
    val io = IO(new Bundle {
        val in  = Input(UInt(16.W))
        val out = Output(Bool())
    })
    io.out := (io.in match {
        // note that case 0.U is an error
        case (0.U) => true.B
        case _   => false.B
    })
}
println(getVerilog(new InputIsZero))

(console):8:16 expected ")"
        case (0.U) => true.B
               ^

: 

## Unapply

실제로 match를 하면 어떤 일이 벌어지나요?
다음과 같은 case class와 멋진 value matching을 수행할 수 있는 이유는 다음과 같습니다.
```scala
case class Something(a: String, b: Int)
val a = Something("A", 3)
a match {
    case Something("A", value) => value
    case Something(str, 3)     => 0
}
```

결과적으로 모든 case class에 대해 생성된 컴패니언 객체에는 **apply** 메서드 외에 **unapply** 메소드도 포함되어 있습니다.
**unapply** 메소드란 무엇일까요?

Scala unapply 메소드는 match 문에 타입을 일치시키고 match 중에 해당 타입에서 **값을 추출**하는 기능을 제공하는 또 다른 형태의 구문 설탕입니다.

다음 예를 살펴보겠습니다.
어떤 이유로 generator가 파이프라인되는 경우 지연이 `3*totalWidth`이고, 그렇지 않으면 지연이 `2*someOtherWidth`라고 가정해 보겠습니다.
case class에는 **unapply**가 정의되어 있으므로 다음과 같이 case class 내부의 값을 일치시킬 수 있습니다.

In [8]:
case class SomeGeneratorParameters(
    someWidth: Int,
    someOtherWidth: Int = 10,
    pipelineMe: Boolean = false
) {
    require(someWidth >= 0)
    require(someOtherWidth >= 0)
    val totalWidth = someWidth + someOtherWidth
}

def delay(p: SomeGeneratorParameters): Int = p match {
    case sg @ SomeGeneratorParameters(_, _, true) => sg.totalWidth * 3
    case SomeGeneratorParameters(_, sw, false) => sw * 2 
}

println(delay(SomeGeneratorParameters(10, 10)))
println(delay(SomeGeneratorParameters(10, 10, true)))

20
60


defined class SomeGeneratorParameters
defined function delay

`delay` 기능을 보면 각 문자의 타입에 대한 일치 외에도 다음과 같은 사항도 있음을 유의해야 합니다.
- 매개변수의 내부 값을 직접 참조
- 때로는 매개변수의 내부 값과 직접 일치합니다.

이는 컴파일러가 `unapply` 메소드를 구현하기 때문에 가능합니다. 대소문자를 적용하지 않는 것은 단지 구문상의 설탕일 뿐입니다. 예를 들어 다음 두 경우의 예는 동일합니다.

```scala
case p: SomeGeneratorParameters => p.sw * 2
case SomeGeneratorParameters(_, sw, _) => sw * 2
```

또한 일치하는 구문과 스타일이 더 많습니다. 다음 두 경우도 동일하지만 두 번째 경우를 사용하면 상위 값을 계속 참조하면서 내부 값을 일치시킬 수 있습니다.

```scala
case SomeGeneratorParameters(_, sw, true) => sw
case sg@SomeGeneratorParameters(_, sw, true) => sw
```

마지막으로 다음과 같은 세 번째 예에서 볼 수 있듯이 조건 검사를 match 문에 직접 포함할 수 있습니다.

```scala
case SomeGeneratorParameters(_, sw, false) => sw * 2
case s@SomeGeneratorParameters(_, sw, false) => s.sw * 2
case s: SomeGeneratorParameters if s.pipelineMe => s.sw * 2
```

이러한 모든 구문은 클래스의 컴패니언 개체에 포함된 Scala unapply 메서드에 의해 활성화됩니다. 클래스를 unapply하고 싶지만 case class로 만들고 싶지 않다면 수동으로 unapply 메소드를 구현할 수 있습니다. 다음 예제는 클래스의 apply 및 unapply 메소드를 수동으로 구현하는 방법을 보여줍니다.

In [50]:
class Boat(val name: String, val length: Int)
object Boat {
    def unapply(b: Boat): Option[(String, Int)] = Some((b.name, b.length))
    def apply(name: String, length: Int): Boat = new Boat(name, length)
}

def getSmallBoats(seq: Seq[Boat]): Seq[Boat] = seq.filter { b =>
    b match {
        case Boat(_, length) if length < 60 => true
        case Boat(_, _) => false
    }
}

val boats = Seq(Boat("Santa Maria", 62), Boat("Pinta", 56), Boat("Nina", 50))
println(getSmallBoats(boats).map(_.name).mkString(" and ") + " are small boats!")

Pinta and Nina are small boats!


defined class Boat
defined object Boat
defined function getSmallBoats
boats: Seq[Boat] = List(
  ammonite.$sess.cmd49$Helper$Boat@45a2c47d,
  ammonite.$sess.cmd49$Helper$Boat@3c15c90c,
  ammonite.$sess.cmd49$Helper$Boat@797a0a85
)

## Partial Functions

이것은 간략한 개요입니다. [이 가이드](https://twitter.github.io/scala_school/pattern-matching-and-functional-composition.html#PartialFunction)에 더 자세한 개요가 있습니다.

부분 함수는 입력의 하위 집합에만 정의된 함수입니다.
옵션과 마찬가지로 부분 함수에는 특정 입력에 대한 값이 없을 수 있습니다.
이것은 `isDefinedAt(...)`로 테스트할 수 있습니다.

Partial functions는 `orElse`와 함께 연결할 수 있습니다.

정의되지 않은 입력으로 `PartialFunction`을 호출하면 런타임 오류가 발생합니다. 예를 들어, `PartialFunction`에 대한 입력이 사용자 정의인 경우 발생할 수 있습니다. 타입 안전성을 높이기 위해 대신 `Option`을 반환하는 함수를 작성하는 것이 좋습니다.

In [51]:
// 이 셀을 조금 덜 지루하게 만드는 도우미 함수.
def printAndAssert(cmd: String, result: Boolean, expected: Boolean): Unit = {
  println(s"$cmd = $result")
  assert(result == expected)
}

// -1, 2, 5 등에 대해 정의
val partialFunc1: PartialFunction[Int, String] = {
  case i if (i + 1) % 3 == 0 => "Something"
}
printAndAssert("partialFunc1.isDefinedAt(2)", partialFunc1.isDefinedAt(2), true)
printAndAssert("partialFunc1.isDefinedAt(5)", partialFunc1.isDefinedAt(5), true)
printAndAssert("partialFunc1.isDefinedAt(1)", partialFunc1.isDefinedAt(1), false)
printAndAssert("partialFunc1.isDefinedAt(0)", partialFunc1.isDefinedAt(0), false)

println(s"partialFunc1(2) = ${partialFunc1(2)}")
try {
  println(partialFunc1(1))
} catch {
  case e: scala.MatchError => println("partialFunc1(0) = can't apply PartialFunctions where they are not defined")
}

// 1, 4, 7 등에 대해 정의
val partialFunc2: PartialFunction[Int, String] = {
  case i if (i + 2) % 3 == 0 => "Something else"
}
printAndAssert("partialFunc2.isDefinedAt(1)", partialFunc2.isDefinedAt(1), true)
printAndAssert("partialFunc2.isDefinedAt(0)", partialFunc2.isDefinedAt(0), false)
println(s"partialFunc2(1) = ${partialFunc2(1)}")
try {
  println(partialFunc2(0))
} catch {
  case e: scala.MatchError => println("partialFunc2(0) = can't apply PartialFunctions where they are not defined")
}

val partialFunc3 = partialFunc1 orElse partialFunc2
printAndAssert("partialFunc3.isDefinedAt(0)", partialFunc3.isDefinedAt(0), false)
printAndAssert("partialFunc3.isDefinedAt(1)", partialFunc3.isDefinedAt(1), true)
printAndAssert("partialFunc3.isDefinedAt(2)", partialFunc3.isDefinedAt(2), true)
printAndAssert("partialFunc3.isDefinedAt(3)", partialFunc3.isDefinedAt(3), false)
println(s"partialFunc3(1) = ${partialFunc3(1)}")
println(s"partialFunc3(2) = ${partialFunc3(2)}")

partialFunc1.isDefinedAt(2) = true
partialFunc1.isDefinedAt(5) = true
partialFunc1.isDefinedAt(1) = false
partialFunc1.isDefinedAt(0) = false
partialFunc1(2) = Something
partialFunc1(0) = can't apply PartialFunctions where they are not defined
partialFunc2.isDefinedAt(1) = true
partialFunc2.isDefinedAt(0) = false
partialFunc2(1) = Something else
partialFunc2(0) = can't apply PartialFunctions where they are not defined
partialFunc3.isDefinedAt(0) = false
partialFunc3.isDefinedAt(1) = true
partialFunc3.isDefinedAt(2) = true
partialFunc3.isDefinedAt(3) = false
partialFunc3(1) = Something else
partialFunc3(2) = Something


defined function printAndAssert
partialFunc1: PartialFunction[Int, String] = <function1>
partialFunc2: PartialFunction[Int, String] = <function1>
partialFunc3: PartialFunction[Int, String] = <function1>

---
# Type Safe Connections<a name="type-safe-connections"></a>

Chisel은 다음을 포함하여 많은 연결 타입을 확인할 수 있습니다.
* Clock에 대한 Bool/UInt

다른 타입의 경우 Chisel을 사용하여 연결할 수 있지만 적절하게 비트를 자르거나 채울 수 있습니다.
* Bool/UInt에서 Bool/UInt로
* Bundle에서 Bundle

In [68]:
class Bundle1 extends Bundle {
  val a = UInt(8.W)
}

class Bundle2 extends Bundle1 {
  val b = UInt(16.W)
}

class BadTypeModule extends Module {
  val io = IO(new Bundle {
    val c  = Input(Clock())
    val in = Input(UInt(2.W))
    val out = Output(Bool())

    val bundleIn = Input(new Bundle2)
    val bundleOut = Output(new Bundle1)
  })
  
  //io.out := io.c // won't work due to different types

  // Okay, but Chisel will truncate the input width to 1 to match the output.
//   io.out := io.in

//   // Compiles; Chisel will connect the common subelements of the two Bundles (in this case, 'a').
//   io.bundleOut := io.bundleIn
}

println(getVerilog(new BadTypeModule))

Elaborating design...
[error] 	at ... ()
[error] 	at ammonite.$sess.cmd67$Helper$BadTypeModule$$anon$1.<init>(cmd67.sc:15)
[error] 	at ammonite.$sess.cmd67$Helper$BadTypeModule.<init>(cmd67.sc:10)
[error] 	at ammonite.$sess.cmd67$Helper.$anonfun$res67_3$1(cmd67.sc:28)
[error] 	at ... ()
[error] 	at ... (Stack trace trimmed to user code only. Rerun with --full-stacktrace to see the full stack trace)


: 

---
# Type Generics<a name="type-generics"></a>
Scala의 제네릭 타입(다형성이라고도 함)은 특히 상속과 결합할 때 매우 복잡합니다.

이 섹션은 당신의 발가락을 젖게 할 것입니다. 자세한 내용은 [본 튜토리얼](https://twitter.github.io/scala_school/type-basics.html)을 확인하세요.

클래스는 타입이 다형성일 수 있습니다. 한 가지 좋은 예는 포함된 타입을 알아야 하는 시퀀스입니다.

In [53]:
val seq1 = Seq("1", "2", "3") // Type is Seq[String]
val seq2 = Seq(1, 2, 3)       // Type is Seq[Int]
val seq3 = Seq(1, "2", true)  // Type is Seq[Any]

seq1: Seq[String] = List("1", "2", "3")
seq2: Seq[Int] = List(1, 2, 3)
seq3: Seq[Any] = List(1, "2", true)

때때로 스칼라 컴파일러는 사용자가 명시적으로 타입을 입력해야 하는 다형성 타입을 결정하는 데 도움이 필요합니다.

In [28]:
//val default = Seq() // Error!
val default = Seq[String]() // User must tell compiler that default is of type Seq[String]
Seq(1, "2", true).foldLeft(default){ (strings, next) =>
    next match {
        case s: String => strings ++ Seq(s)
        case _ => strings
    }
}

default: Seq[String] = List()
res27_1: Seq[String] = List("2")

함수는 입력 또는 출력 타입에서 다형성일 수도 있습니다. 다음 예제에서는 코드 블록을 실행하는 데 걸리는 시간을 계산하는 함수를 정의합니다. 코드 블록의 반환 타입에 따라 매개변수화됩니다. *`=> T` 구문은 인수 목록이 없는 익명 함수를 인코딩합니다. `{ ... }` 대 `{ x => ... }`.*

In [29]:
def time[T](block: => T): T = {
    val t0 = System.nanoTime()
    val result = block
    val t1 = System.nanoTime()
    val timeMillis = (t1 - t0) / 1000000.0
    println(s"Block took $timeMillis milliseconds!")
    result
}

// Adds 1 through a million
val int = time { (1 to 1000000).reduce(_ + _) }
println(s"Add 1 through a million is $int")

// Finds the largest number under a million that, in hex, contains "beef"
val string = time {
    (1 to 1000000).map(_.toHexString).filter(_.contains("beef")).last
}
println(s"The largest number under a million that has beef: $string")

Block took 8.099028 milliseconds!
Add 1 through a million is 1784293664
Block took 49.223639 milliseconds!
The largest number under a million that has beef: ebeef


defined function time
int: Int = 1784293664
string: String = "ebeef"

## Chisel Type Hierarchy
Chisel을 사용하여 타입 제네릭 코드를 작성하려면 Chisel의 타입 계층에 대해 약간 아는 것이 도움이 됩니다.

'chisel3.Data'는 Chisel 하드웨어 타입의 기본 클래스입니다.
`UInt`, `SInt`, `Vec`, `Bundle` 등은 모두 `Data`의 인스턴스입니다.
`Data`는 IO에서 사용할 수 있으며 `:=`, 전선, reg 등을 지원합니다.

레지스터는 Chisel의 다형성 코드의 좋은 예입니다.
`RegEnable`(`Bool` 활성화 신호가 있는 레지스터)의 구현을 살펴보세요. [여기](https://github.com/freechipsproject/chisel3/blob/v3.0.0/src/main/scala/chisel3/util/Reg.scala#L10).
apply function은 `[T <: Data]`용으로 템플릿화되어 있습니다. 즉, `RegEnable`은 모든 Chisel 하드웨어 타입에서 작동합니다.

일부 작업은 '비트'의 하위 타입에만 정의됩니다(예: '+').
이것이 `UInt` 또는 `SInt`를 추가할 수 있지만 `Bundle` 또는 `Vec`을 추가할 수 없는 이유입니다.

<span style="color:blue">**Example: Type Generic ShiftRegister**<a name="type-generic-shift-register"></a></span><br>
Scala에서 객체와 함수는 매개변수로 취급할 수 있는 유일한 것이 아닙니다. 타입을 매개변수로 취급할 수도 있습니다.

일반적으로 타입 제약 조건을 제공해야 합니다. 이 경우 객체를 번들에 넣고 연결(:=)하고 ​​레지스터를 생성(RegNext)할 수 있기를 원합니다. 이러한 작업은 임의의 개체에서 수행할 수 없습니다. 예를 들어 wire := 3은 Chisel UInt가 아니라 Scala Int이기 때문에 불법입니다. 타입 제약을 사용하여 타입 T가 Data의 하위 클래스라고 말하면, 모든 Data에 대해 :=가 정의되어 있기 때문에 타입 T의 모든 객체에 :=를 사용할 수 있습니다.

다음은 타입을 매개변수로 사용하는 시프트 레지스터의 구현입니다. *gen*은 사용할 너비를 알려주는 타입 T의 인수입니다. 예를 들어 new ShiftRegister(UInt(4.W))는 4비트 UInt용 시프트 레지스터입니다.
*gen*은 또한 스칼라 컴파일러가 타입 T-를 추론할 수 있도록 합니다.
더 구체적이고 싶다면 new ShiftRegister[UInt](UInt(4.W)) 를 작성할 수 있습니다.
그러나 Scala 컴파일러는 [UInt]를 생략해도 알아낼 만큼 충분히 똑똑합니다.

In [30]:
class ShiftRegisterIO[T <: Data](gen: T, n: Int) extends Bundle {
    require (n >= 0, "Shift register must have non-negative shift")
    
    val in = Input(gen)
    val out = Output(Vec(n + 1, gen)) // + 1 because in is included in out
    override def cloneType: this.type = (new ShiftRegisterIO(gen, n)).asInstanceOf[this.type]
}

class ShiftRegister[T <: Data](gen: T, n: Int) extends Module {
    val io = IO(new ShiftRegisterIO(gen, n))
    
    io.out.foldLeft(io.in) { case (in, out) =>
        out := in
        RegNext(in)
    }
}

visualize(() => new ShiftRegister(SInt(6.W), 3))
test(new ShiftRegister(SInt(6.W), 3)) { c => 
    println(s"Testing ShiftRegister of type ${c.io.in} and depth ${c.io.out.length}")
    for (i <- 0 until 10) {
        c.io.in.poke(i.S) // magic literal creation
        println(s"$i: ${c.io.out.indices.map { index => c.io.out(index).peek().litValue} }")
        c.clock.step(1)
    }}

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<?xml-stylesheet href="styles.css" type="text/css"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.40.1 (0)
 -->
<!-- Title: ShiftRegister Pages: 1 -->
 
 
 ShiftRegister 
 
 
 cluster_ShiftRegister 
 
 ShiftRegister 
 
<!-- cluster_ShiftRegister_clock -->
 
 cluster_ShiftRegister_clock 
 
 clock 
 
<!-- cluster_ShiftRegister_reset -->
 
 cluster_ShiftRegister_reset 
 
 reset 
 
<!-- cluster_ShiftRegister_io_in -->
 
 cluster_ShiftRegister_io_in 
 
 io_in 
 
<!-- cluster_ShiftRegister_io_out_0 -->
 
 cluster_ShiftRegister_io_out_0 
 
 io_out_0 
 
<!-- cluster_ShiftRegister_io_in->cluster_ShiftRegister_io_out_0 -->
 
 cluster_ShiftRegister_io_in->cluster_ShiftRegister_io_out_0 
 
 
 
<!-- struct_cluster_ShiftRegister_REG -->
 
 struct_cluster_ShiftRegister_REG 
 
 
 ● 
 
 REG 
 
 ● 
 
<!-- cluster_ShiftRegister_io_in->struct_cluster_ShiftRegister_REG -->
 
 cluster_ShiftRegister_io_in->struct_cluster_ShiftRegister_REG:in 
 
 
 
<!-- cluster_ShiftRegister_io_out_1 -->
 
 cluster_ShiftRegister_io_out_1 
 
 io_out_1 
 
<!-- cluster_ShiftRegister_io_out_2 -->
 
 cluster_ShiftRegister_io_out_2 
 
 io_out_2 
 
<!-- cluster_ShiftRegister_io_out_3 -->
 
 cluster_ShiftRegister_io_out_3 
 
 io_out_3 
 
<!-- struct_cluster_ShiftRegister_REG->cluster_ShiftRegister_io_out_1 -->
 
 struct_cluster_ShiftRegister_REG:out->cluster_ShiftRegister_io_out_1 
 
 
 
<!-- struct_cluster_ShiftRegister_REG_1 -->
 
 struct_cluster_ShiftRegister_REG_1 
 
 
 ● 
 
 REG_1 
 
 ● 
 
<!-- struct_cluster_ShiftRegister_REG->struct_cluster_ShiftRegister_REG_1 -->
 
 struct_cluster_ShiftRegister_REG:out->struct_cluster_ShiftRegister_REG_1:in 
 
 
 
<!-- struct_cluster_ShiftRegister_REG_1->cluster_ShiftRegister_io_out_2 -->
 
 struct_cluster_ShiftRegister_REG_1:out->cluster_ShiftRegister_io_out_2 
 
 
 
<!-- struct_cluster_ShiftRegister_REG_2 -->
 
 struct_cluster_ShiftRegister_REG_2 
 
 
 ● 
 
 REG_2 
 
 ● 
 
<!-- struct_cluster_ShiftRegister_REG_1->struct_cluster_ShiftRegister_REG_2 -->
 
 struct_cluster_ShiftRegister_REG_1:out->struct_cluster_ShiftRegister_REG_2:in 
 
 
 
<!-- struct_cluster_ShiftRegister_REG_2->cluster_ShiftRegister_io_out_3 -->
 
 struct_cluster_ShiftRegister_REG_2:out->cluster_ShiftRegister_io_out_3

Elaborating design...
Done elaborating.
Testing ShiftRegister of type SInt<6>(IO io_in in ShiftRegister) and depth 4
0: Vector(0, 0, 0, 0)
1: Vector(1, 0, 0, 0)
2: Vector(2, 1, 0, 0)
3: Vector(3, 2, 1, 0)
4: Vector(4, 3, 2, 1)
5: Vector(5, 4, 3, 2)
6: Vector(6, 5, 4, 3)
7: Vector(7, 6, 5, 4)
8: Vector(8, 7, 6, 5)
9: Vector(9, 8, 7, 6)
test ShiftRegister Success: 0 tests passed in 12 cycles in 0.024947 seconds 481.02 Hz


defined class ShiftRegisterIO
defined class ShiftRegister

일반적으로 타입 제네릭과 함께 상속을 사용하지 않는 것이 좋습니다.
제대로 하는 것은 매우 까다로울 수 있고 빨리 좌절할 수 있습니다.

## Type Generics with Typeclasses

위의 예는 `:=` 또는 `RegNext()`와 같은 `Data`의 모든 인스턴스에서 수행할 수 있는 간단한 작업으로 제한되었습니다.
DSP 회로를 생성할 때 덧셈 및 곱셈과 같은 수학적 연산을 수행하고 싶습니다.
`dsptools` 라이브러리는 타입 매개변수화된 DSP generator를 작성하기 위한 도구를 제공합니다.

다음은 곱하기 누산 모듈을 작성하는 예입니다.
이는 `FixedPoint`, `SInt` 또는 `DspComplex[T]`(`dsptools`에서 제공하는 복소수 타입)에 대한 MAC(multiply-accumulate)를 생성하는 데 사용할 수 있습니다.
`dsptools`가 타입 클래스를 사용하기 때문에 타입 바인딩의 구문이 약간 다릅니다.
그것들은 이 노트북의 범위를 벗어납니다.
typeclasses 사용에 대한 자세한 내용은 `dsptools` readme 및 문서를 참조하십시오.

`T <: Data : Ring`은 `T`가 `Data`의 하위 타입이자 `Ring`임을 의미합니다.
'링'은 'dsptools'에서 '+' 및 '*'(다른 작업 중에서)가 있는 숫자로 정의됩니다.

_'Ring'의 대안은 'Real'이지만 복소수가 'Real'이 아니기 때문에 'DspComplex()'에 대한 MAC을 만들 수 없습니다._



In [31]:
import chisel3.experimental._
import dsptools.numbers._

class Mac[T <: Data : Ring](genIn : T, genOut: T) extends Module {
    val io = IO(new Bundle {
        val a = Input(genIn)
        val b = Input(genIn)
        val c = Input(genIn)
        val out = Output(genOut)
    })
    io.out := io.a * io.b + io.c
}

println(getVerilog(new Mac(UInt(4.W), UInt(6.W)) ))
println(getVerilog(new Mac(SInt(4.W), SInt(6.W)) ))
println(getVerilog(new Mac(FixedPoint(4.W, 3.BP), FixedPoint(6.W, 4.BP))))


Elaborating design...
Done elaborating.
module Mac(
  input        clock,
  input        reset,
  input  [3:0] io_a,
  input  [3:0] io_b,
  input  [3:0] io_c,
  output [5:0] io_out
);
  wire [7:0] _T = io_a * io_b; // @[UIntTypeClass.scala 40:41]
  wire [7:0] _GEN_0 = {{4'd0}, io_c}; // @[UIntTypeClass.scala 19:40]
  wire [7:0] _T_2 = _T + _GEN_0; // @[UIntTypeClass.scala 19:40]
  assign io_out = _T_2[5:0]; // @[cmd30.sc 11:12]
endmodule

Elaborating design...
Done elaborating.
module Mac(
  input        clock,
  input        reset,
  input  [3:0] io_a,
  input  [3:0] io_b,
  input  [3:0] io_c,
  output [5:0] io_out
);
  wire [7:0] _T = $signed(io_a) * $signed(io_b); // @[SIntTypeClass.scala 45:41]
  wire [7:0] _GEN_0 = {{4{io_c[3]}},io_c}; // @[SIntTypeClass.scala 19:40]
  wire [7:0] _T_3 = $signed(_T) + $signed(_GEN_0); // @[SIntTypeClass.scala 19:40]
  assign io_out = _T_3[5:0]; // @[cmd30.sc 11:12]
endmodule

Elaborating design...
Done elaborating.
module Mac(
  input        clock,

import chisel3.experimental._

import dsptools.numbers._


defined class Mac

<span style="color:red">**Exercise: Mac as Object**</span><br>

Mac '모듈'에는 적은 수의 입력과 하나의 출력만 있습니다.
다른 Chisel generator가 다음과 같은 코드를 작성하는 것이 편리할 수 있습니다.
```scala
val out = Mac(a, b, c)
```

`Mac` 기능을 구현하는 아래의 `Mac` 도우미 개체에서 `Apply` 메서드를 구현합니다.

In [32]:
object Mac {
    def apply[T <: Data : Ring](a: T, b: T, c: T): T = {
        ??? // your code
    }
}

class MacTestModule extends Module {
    val io = IO(new Bundle {
        val uin = Input(UInt(4.W))
        val uout = Output(UInt())
        val sin = Input(SInt(4.W))
        val sout = Output(SInt())
        //val fin = Input(FixedPoint(16.W, 12.BP))
        //val fout = Output(FixedPoint())
    })
    // for each IO pair, do out = in * in + in
    io.uout := Mac(io.uin, io.uin, io.uin)
    io.sout := Mac(io.sin, io.sin, io.sin)
    //io.fout := Mac(io.fin, io.fin, io.fin)
}
println(getVerilog(new MacTestModule))

Elaborating design...
[error] 	at ... ()
[error] 	at ammonite.$sess.cmd31$Helper$Mac$.apply(cmd31.sc:3)
[error] 	at ammonite.$sess.cmd31$Helper$MacTestModule.$anonfun$new$1(cmd31.sc:17)
[error] 	at chisel3.Data.$anonfun$$colon$eq$1(Data.scala:541)
[error] 	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
[error] 	at chisel3.internal.prefix$.apply(prefix.scala:32)
[error] 	at chisel3.Data.$colon$eq(Data.scala:541)
[error] 	at ammonite.$sess.cmd31$Helper$MacTestModule.<init>(cmd31.sc:17)
[error] 	at ammonite.$sess.cmd31$Helper$.$anonfun$res31_2$1(cmd31.sc:21)
[error] 	at ... ()
[error] 	at ... (Stack trace trimmed to user code only. Rerun with --full-stacktrace to see the full stack trace)


: 

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong> (click to toggle displaying)</label>
<article>
<pre style="background-color:#f7f7f7">

        a * b + c

</pre></article></div></section></div>

<span style="color:red">**Exercise: Integrator**</span><br>
아래 그림과 같이 통합자를 구현합니다. $n_1$은 `genReg`의 너비이고 $n_2$는 `genIn`의 너비입니다.

`Reg`, `RegInit`, `RegNext`, `RegEnable` 등이 `T <: Data` 타입에 대해 템플릿화된다는 것을 잊지 마십시오.

<img src="images/integrator.svg" alt="Integrator" style="width: 250px;"/>

In [ ]:
class Integrator[T <: Data : Ring](genIn: T, genReg: T) extends Module {
    val io = IO(new Bundle {
        val in  = Input(genIn)
        val out = Output(genReg)
    })
    
    ??? // your code
}

test(new Integrator(SInt(4.W), SInt(8.W))) { c =>
    c.io.in.poke(3.S)
    c.io.out.expect(0.S)
    c.clock.step(1)
    c.io.in.poke(-4.S)
    c.io.out.expect(3.S)
    c.clock.step(1)
    c.io.in.poke(6.S)
    c.io.out.expect(-1.S)
    c.clock.step(1)
    c.io.out.expect(5.S)
}

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-2" />
<label for="check-2"><strong>Solution</strong> (click to toggle displaying)</label>
<article>
<pre style="background-color:#f7f7f7">

class Integrator\[T <: Data : Ring\](genIn: T, genReg: T) extends Module {
    val io = IO(new Bundle {
        val in  = Input(genIn.cloneType)
        val out = Output(genReg.cloneType)
    })
    
    val reg = RegInit(genReg, Ring[T].zero) // init to zero
    reg := reg + io.in
    io.out := reg
}

</pre></article></div></section></div>

---
# Creating a Custom Type<a name="creating-a-custom-type"></a>

Chisel을 강력하게 만드는 것 중 하나는 확장성입니다.
애플리케이션에 맞게 조정된 고유한 작업 및 표현이 있는 고유한 타입을 추가할 수 있습니다.
이 섹션에서는 사용자 정의 타입을 만드는 방법을 소개합니다.

<span style="color:blue">**Example: DspComplex**</span><br>
`DspComplex`는 **dsptools** [여기](https://github.com/ucb-bar/dsptools/blob/v1.0.0/src/main/scala/dsptools/numbers/chisel_concrete)에 정의된 사용자 지정 데이터 타입입니다. /DspComplex.scala#L59).
이해해야 할 핵심 라인은 다음과 같습니다.
```scala
class DspComplex[T <: Data:Ring](val real: T, val imag: T) extends Bundle { ... }
```
'DspComplex'는 타입 제네릭 컨테이너입니다.
즉, 복소수의 실수 부분과 허수 부분은 `T <: Data : Ring`에 지정된 형식 제약 조건을 충족하는 한 모든 형식이 될 수 있습니다.

`T <: Data`는 `T`가 Chisel 객체의 기본 타입인 `chisel3.Data`의 하위 타입임을 의미합니다.
즉, 'DspComplex'는 임의의 Scala 타입이 아닌 Chisel 타입인 객체에만 작동합니다.

'T : Ring'은 'T'에 대한 Ring 타입 클래스 구현이 존재함을 의미합니다.
`Ring` 타입 클래스는 `+` 및 `*` 연산자와 덧셈 및 곱셈 ID를 정의합니다(고리에 대한 자세한 내용은 [이 Wikipedia 기사](https://en.wikipedia.org/wiki/Ring_(mathematics)) 참조). .
**dsptools**는 [여기](https://github.com/ucb-bar/dsptools/tree/v1.0.0/src/main/scala/dsptools/numbers/chisel_types)에서 일반적으로 사용되는 치즐 타입에 대한 타입 클래스를 정의합니다.

**dsptools**는 또한 `DspComplex`에 대한 `Ring` 타입 클래스를 정의하므로 복소수로 MAC generator를 재사용할 수 있습니다.

In [33]:
println(getVerilog(new Mac(DspComplex(SInt(4.W), SInt(4.W)), DspComplex(SInt(6.W), SInt(6.W))) ))

Elaborating design...
Done elaborating.
module Mac(
  input        clock,
  input        reset,
  input  [3:0] io_a_real,
  input  [3:0] io_a_imag,
  input  [3:0] io_b_real,
  input  [3:0] io_b_imag,
  input  [3:0] io_c_real,
  input  [3:0] io_c_imag,
  output [5:0] io_out_real,
  output [5:0] io_out_imag
);
  wire [3:0] _T_2 = $signed(io_b_real) + $signed(io_b_imag); // @[SIntTypeClass.scala 19:40]
  wire [3:0] _T_5 = $signed(io_a_real) + $signed(io_a_imag); // @[SIntTypeClass.scala 19:40]
  wire [3:0] _T_8 = $signed(io_a_imag) - $signed(io_a_real); // @[SIntTypeClass.scala 29:50]
  wire [7:0] _T_9 = $signed(io_a_real) * $signed(_T_2); // @[SIntTypeClass.scala 45:41]
  wire [7:0] _T_10 = $signed(_T_5) * $signed(io_b_imag); // @[SIntTypeClass.scala 45:41]
  wire [7:0] _T_11 = $signed(_T_8) * $signed(io_b_real); // @[SIntTypeClass.scala 45:41]
  wire [7:0] _T_14 = $signed(_T_9) - $signed(_T_10); // @[SIntTypeClass.scala 29:50]
  wire [7:0] _T_17 = $signed(_T_9) + $signed(_T_11); // @[SI

<span style="color:red">**Exercise: Sign-magnitude Numbers**</span><br>
부호 크기 표현을 사용하고 모든 DSP generator를 재사용하려고 한다고 가정합니다.
Typeclass는 이러한 종류의 임시 다형성을 가능하게 합니다.
다음 예제는 SignMagnitude 타입 구현의 시작과 해당 타입을 Mac generator와 함께 사용할 수 있도록 하는 `Ring` 타입 클래스의 구현을 제공합니다.

`+` 및 `*`에 대한 구현을 입력합니다.
`unary_-()`에 대한 구현 후에 패턴화해야 합니다.
다음 블록에는 'SignMagnitude'를 사용하는 'Mac'의 정확성을 확인하는 테스트가 포함되어 있습니다.

In [33]:
class SignMagnitude(val magnitudeWidth: Option[Int] = None) extends Bundle {
    val sign = Bool()
    val magnitude = magnitudeWidth match {
        case Some(w) => UInt(w.W)
        case None    => UInt()
    }
    def +(that: SignMagnitude): SignMagnitude = {
        // Implement this!
    }
    def -(that: SignMagnitude): SignMagnitude = {
        this.+(-that)
    }
    def unary_-(): SignMagnitude = {
        val result = Wire(new SignMagnitude())
        result.sign := !this.sign
        result.magnitude := this.magnitude
        result
    }
    def *(that: SignMagnitude): SignMagnitude = {
        // Implement this!
    }
}
trait SignMagnitudeRing extends Ring[SignMagnitude] {
    def plus(f: SignMagnitude, g: SignMagnitude): SignMagnitude = {
        f + g
    }
    def times(f: SignMagnitude, g: SignMagnitude): SignMagnitude = {
        f * g
    }
    def one: SignMagnitude = {
        val one = Wire(new SignMagnitude(Some(1)))
        one.sign := false.B
        one.magnitude := 1.U
        one
    }
    def zero: SignMagnitude = {
        val zero = Wire(new SignMagnitude(Some(0)))
        zero.sign := false.B
        zero.magnitude := 0.U
        zero
    }
    def negate(f: SignMagnitude): SignMagnitude = {
        -f
    }
    
    // Leave unimplemented for this example
    def minusContext(f: SignMagnitude, g: SignMagnitude): SignMagnitude = ???
    def negateContext(f: SignMagnitude): SignMagnitude = ???
    def plusContext(f: SignMagnitude,g: SignMagnitude): SignMagnitude = ???
    def timesContext(f: SignMagnitude,g: SignMagnitude): SignMagnitude = ???
}
implicit object SignMagnitudeRingImpl extends SignMagnitudeRing

cmd33.sc:7: type mismatch;
 found   : Unit
 required: Helper.this.SignMagnitude
 Note: implicit object SignMagnitudeRingImpl is not applicable here because it comes after the application point and it lacks an explicit result type
    def +(that: SignMagnitude): SignMagnitude = {
                                                ^cmd33.sc:19: type mismatch;
 found   : Unit
 required: Helper.this.SignMagnitude
 Note: implicit object SignMagnitudeRingImpl is not applicable here because it comes after the application point and it lacks an explicit result type
    def *(that: SignMagnitude): SignMagnitude = {
                                                ^Compilation Failed

: 

In [33]:
import chisel3.experimental.BundleLiterals._

test(new Mac(new SignMagnitude(Some(4)), new SignMagnitude(Some(5)))) { c =>
    c.io.a.poke(chiselTypeOf(c.io.a).Lit(_.sign -> false.B, _.magnitude -> 3.U))
    c.io.b.poke(chiselTypeOf(c.io.b).Lit(_.sign -> false.B, _.magnitude -> 3.U))
    c.io.c.poke(chiselTypeOf(c.io.c).Lit(_.sign -> false.B, _.magnitude -> 2.U))
    c.io.out.expect(chiselTypeOf(c.io.out).Lit(_.sign -> false.B, _.magnitude -> 11.U))

    c.io.c.sign.poke(true.B)
    c.io.out.expect(chiselTypeOf(c.io.out).Lit(_.sign -> false.B, _.magnitude -> 7.U))

    c.io.b.sign.poke(true.B)
    c.io.out.expect(chiselTypeOf(c.io.out).Lit(_.sign -> true.B, _.magnitude -> 11.U))
}
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

cmd33.sc:3: not found: type SignMagnitude
val res33_1 = test(new Mac(new SignMagnitude(Some(4)), new SignMagnitude(Some(5)))) { c =>
                               ^cmd33.sc:3: not found: type SignMagnitude
val res33_1 = test(new Mac(new SignMagnitude(Some(4)), new SignMagnitude(Some(5)))) { c =>
                                                           ^Compilation Failed

: 

출력이 합리적인지 확인하려면 Verilog를 살펴보세요.

In [33]:
println(getVerilog(new Mac(new SignMagnitude(Some(4)), new SignMagnitude(Some(5)))))

cmd33.sc:1: not found: type SignMagnitude
val res33 = println(getVerilog(new Mac(new SignMagnitude(Some(4)), new SignMagnitude(Some(5)))))
                                           ^cmd33.sc:1: not found: type SignMagnitude
val res33 = println(getVerilog(new Mac(new SignMagnitude(Some(4)), new SignMagnitude(Some(5)))))
                                                                       ^Compilation Failed

: 

'SignMagnitude'는 'DspComplex'에서도 작동합니다!

In [33]:
println(getVerilog(new Mac(DspComplex(new SignMagnitude(Some(4)), new SignMagnitude(Some(4))), DspComplex(new SignMagnitude(Some(5)), new SignMagnitude(Some(5))))))

cmd33.sc:1: not found: type SignMagnitude
val res33 = println(getVerilog(new Mac(DspComplex(new SignMagnitude(Some(4)), new SignMagnitude(Some(4))), DspComplex(new SignMagnitude(Some(5)), new SignMagnitude(Some(5))))))
                                                      ^cmd33.sc:1: not found: type SignMagnitude
val res33 = println(getVerilog(new Mac(DspComplex(new SignMagnitude(Some(4)), new SignMagnitude(Some(4))), DspComplex(new SignMagnitude(Some(5)), new SignMagnitude(Some(5))))))
                                                                                  ^cmd33.sc:1: not found: type SignMagnitude
val res33 = println(getVerilog(new Mac(DspComplex(new SignMagnitude(Some(4)), new SignMagnitude(Some(4))), DspComplex(new SignMagnitude(Some(5)), new SignMagnitude(Some(5))))))
                                                                                                                          ^cmd33.sc:1: not found: type SignMagnitude
val res33 = println(getVerilog(new Mac(D

: 

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-3" />
<label for="check-3"><strong>Solution</strong> (click to toggle displaying)</label>
<article>
<pre style="background-color:#f7f7f7">
    // implementations for class SignMagnitude

    def +(that: SignMagnitude): SignMagnitude = {
      val result = Wire(new SignMagnitude())
      val signsTheSame = this.sign === that.sign
      when (signsTheSame) {
        result.sign      := this.sign
        result.magnitude := this.magnitude + that.magnitude
      } .otherwise {
        when (this.magnitude > that.magnitude) {
          result.sign      := this.sign
          result.magnitude := this.magnitude - that.magnitude
        } .otherwise {
          result.sign      := that.sign
          result.magnitude := that.magnitude - this.magnitude
        }   
      }   
      result
    }
    def *(that: SignMagnitude): SignMagnitude = {
        val result = Wire(new SignMagnitude())
        result.sign := this.sign ^ that.sign
        result.magnitude := this.magnitude * that.magnitude
        result
    }


</pre></article></div></section></div>